In [2]:
import numpy as np 
from scipy.optimize import fsolve

# DEALING WITH INVENTORY VARIATION (1) Starting close enough to the equilibrium

$$ \dot{V} = Y - [\Gamma^T Y] - \Xi^T I^r - C $$ 

With : 
$$Y = K/\nu$$
$$L = K/a$$
$$\Pi = pY  - w L- Y [\Gamma p] - \delta K  [\Xi p]$$
$$I^r = \delta K + \Pi$$ 
$$C = C^{pond} \sum(w L)$$

What this program does is, given : $\lambda$,$w$, $\Gamma$,$\Xi$,$C^{pond}$,$\delta$,$\nu$,$p$,$a$ returns the value of $K$ so that we have market-clearing conditions respecting the employment


In [16]:

def dotV_for_K(K,params):
    ### Unpacking
    Gamma   =params['Gamma']
    Xi      =params['Xi']
    Cpond   =params['Cpond']
    delta   =params['delta']
    nu      =params['nu']
    p       =params['p']
    a       =params['a']
    w       =params['w']

    #### Deriving equations 
    Y = K/nu 
    L = K/a
    Inter = np.matmul(np.transpose(Gamma),Y) 
    C = Cpond*sum(w*L)/p
    Pi = p*Y - w*L- Y*np.matmul(Gamma,p) - delta*K* np.matmul(Xi,p)
    Ir = Pi/(np.matmul(Xi,p)) + K*delta
    
    ### Returning dotV
    return Y - Inter - C - np.matmul(np.transpose(Xi),Ir)


In [29]:
# TYPICAL 4-SECTORAL intermediate consumption
# sector 1 consumption 
# sector 2 good for sector 1 
# sector 3 good for sector 2 
# sector 4 Capital

############# PARAMETERS ################
dic4 = {}    
dic4['Gamma'] = np.array([  [0,.1,0 , 0],
                           [0, 0,.1, 0],
                           [0, 0, 0, 0],
                           [0, 0, 0, 0]]) 
dic4['Xi']    = np.array([  [0,0,0,1],
                           [0,0,0,1],
                           [0,0,0,1],
                           [0,0,0,1]])
dic4['Cpond'] = np.array([1  ,  0,  0,  0])
dic4['delta'] = np.array([.05,.05,.05,.05])
dic4['nu']    = np.array([3.,3.,3.,3.])
dic4['p']     = np.array([1.,1.,1.,1.])
dic4['a']     = np.array([3 ,3 ,3 ,3 ])
dic4['w']     = np.array([.5,.5,.5,.5])  

dic4['employment'] = .9
#dic4['N']          = 1   
dic4['GDP'] = 100

############ FINDING RELATIVE SIZE ######
K = fsolve(dotV_for_K,np.array([1.,1.,1.,1.]),args=dic4)

############ SCALING (One or the other)
# Population
if 'N' in dic4.keys():
    L = np.sum(K/dic4['a'])
    Scale = L/dic4['employment']
    K*=dic4['N']/Scale
    dic4['GDP']=sum(dic4['p']*(K/dic4['nu'] - np.matmul(np.transpose(dic4['Gamma']),K/dic4['nu']) ) )

# GDP
else:
    GDPN = sum(dic4['p']*(K/dic4['nu'] - np.matmul(np.transpose(dic4['Gamma']),K/dic4['nu']) ) ) #nominal GDP
    K*= dic4['GDP']/GDPN
    dic4['N'] = np.sum(K/dic4['a'])/dic4['employment']

########### DEDUCING SOME QUANTITIES
dic4['omega']= np.sum(dic4['w']*K/dic4['a'])/dic4['GDP']
print(dic4['omega'])

############ CHECKING QUALITY
print(K,dotV_for_K(K,dic4))

2.2500000033725307
0.5291005291005291
[158.73015873  15.87301587   1.58730159 141.26984127] [ 7.10542736e-15  0.00000000e+00  3.33066907e-16 -7.10542736e-15]


In [61]:
Nsect = 10 
dicN = {} 
dicN['Gamma'] = 0.05*np.random.lognormal(size=(Nsect,Nsect))   
dicN['Xi']    = 0.01*np.random.lognormal(size=(Nsect,Nsect)) 
dicN['Cpond'] = np.random.lognormal(size=Nsect) 
dicN['Cpond']/= np.sum(dicN['Cpond'])
dicN['delta'] = np.array([.05]*Nsect)
dicN['nu']    = np.array([3.]*Nsect)
dicN['p']     = np.array([1.]*Nsect)
dicN['a']     = np.array([3 ]*Nsect)
dicN['w']     = np.array([.5]*Nsect)

dicN['employment'] = .9
dicN['N']     = 1

LeontievInverse = np.linalg.inv(np.eye(Nsect)-dicN['Gamma']-dicN['delta']*dicN['nu']*dicN['Xi'])
Eigenvalues,Eigenvectors = np.linalg.eig(LeontievInverse)
print('Eigenvalues',np.real(Eigenvalues))

############ FINDING RELATIVE SIZE ######
K = fsolve(dotV_for_K,np.array([1.]*Nsect),args=dicN)

############ SCALING (One or the other)
# Population
if 'N' in dicN.keys():
    L = np.sum(K/dicN['a'])
    Scale = L/dicN['employment']
    K*=dicN['N']/Scale
    dicN['GDP']=sum(dicN['p']*(K/dicN['nu'] - np.matmul(np.transpose(dicN['Gamma']),K/dicN['nu']) ) )

# GDP
else:
    GDPN = sum(dicN['p']*(K/dicN['nu'] - np.matmul(np.transpose(dicN['Gamma']),K/dicN['nu']) ) ) #nominal GDP
    K*= dicN['GDP']/GDPN
    dicN['N'] = np.sum(K/dicN['a'])/dicN['employment']

print('Capital',K)

Eigenvalues [4.90316189 1.41768562 1.09055593 1.09055593 0.83152452 1.07590257
 0.92219071 0.92219071 0.99954478 0.94598401]
Capital [0.33354162 0.26419526 0.2319804  0.3057239  0.23771194 0.1565128
 0.3273572  0.14821592 0.18667007 0.50809088]
[[0.03821992 0.05579551 0.0737206  0.3679107  0.14745124 0.027548
  0.01486963 0.10608457 0.13055444 0.01792438]
 [0.05154683 0.13461918 0.04756743 0.08068228 0.02370134 0.01740005
  0.12326797 0.02120806 0.0097295  0.09643878]
 [0.07632014 0.14738301 0.37750063 0.10456468 0.00129465 0.0729341
  0.05473039 0.03005931 0.01737898 0.08982053]
 [0.04617291 0.08119389 0.08993386 0.05234479 0.01657395 0.04602918
  0.03907904 0.03417651 0.09091223 0.01089054]
 [0.12154231 0.2256873  0.0720355  0.04676909 0.03480964 0.01657987
  0.024443   0.06577225 0.05696694 0.06771567]
 [0.11064238 0.09289232 0.04586527 0.08980033 0.0212329  0.044278
  0.0156889  0.02428852 0.44871828 0.10484106]
 [0.02530092 0.08266739 0.07534245 0.15581028 0.09439027 0.02265451
  

In [55]:
# Monosectoral
Gamma = np.array([0.1])
Xi = np.array([1. ])
Cpond = np.array([1.]) 
delta = np.array([0.05])
nu =np.array([3. ])
p = np.array([1.])
w=np.array([.6])


In [124]:
# TYPICAL 3-SECTORAL
from scipy.optimize import fsolve

Gamma = np.array([  [0,.1,0],
                    [0,.1,0],
                    [0,.1,0]])
Xi    = np.array([  [0,0,1],
                    [0,0,1],
                    [0,0,1]])
Cpond = np.array([1,0,0])
delta = np.array([0.05,0.05,0.05])
nu    = np.array([3.,3.,3.])
p     = np.array([1.05,1.,1.])
a     = np.array([3,2,1])
w     = np.array([0.4,0.5,.6])  
Kini = np.array([1.,1.,1.])
employment = .9

K = fsolve(dotV_for_K,Kini,args=[Gamma,Xi,Cpond,delta,nu,p,a,w])
L = np.sum(K/a)
Neq = L/employment
K/=Neq
print(K,dotV_for_K(K,[Gamma,Xi,Cpond,delta,nu,p,a,w]))

[1.27364185 0.17907445 0.38631791] [-0.00057488 -0.00160966  0.00019163]


c:\ProgramData\Anaconda3\envs\py3.11\Lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [129]:
# TYPICAL 3-SECTORAL
from scipy.optimize import fsolve

Gamma = np.array([  [0,.1,0],
                    [0,.1,0],
                    [0,.1,0]]) 
#Gamma = 0.1*np.random.lognormal(size=(3,3))
#print(Gamma)
Xi    = np.array([  [0,0,1],
                    [0,0,1],
                    [0,0,1]])
                    
Cpond = np.array([1,0,0])
delta = np.array([0.05,0.05,0.05])
nu    = np.array([3.,3.,3.])
p     = np.array([1.05,1.,1.])
a     = np.array([3,2,1])
w     = np.array([0.4,0.5,.6])  
Kini = np.array([1.,1.,1.])
employment = .9

K = fsolve(dotV_for_K,Kini)
L = np.sum(K/a)
Neq = L/employment
K/=Neq
print(K,dotV_for_K(K))

[[0.32792069 0.11706981 0.0219704 ]
 [0.06907997 0.03987691 0.36557226]
 [0.01772603 0.03077198 0.19067264]]
[1.77272727 0.29545455 0.18181818] [-0.01009051  0.02351499 -0.02652091]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 5)